In [34]:
# imports
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from tqdm.notebook import tqdm
# globals
data_path  =os.path.join(os.getcwd(),'data')
train_path =os.path.join(data_path,'Train')
test_path  =os.path.join(data_path,'Test')

# data labeling and formattimg 
'''
    for each folder map the image from 0 to upperlimit
'''
dlist=os.listdir(train_path)
dlist=[int(d) for d in dlist]
dlist.sort()
lower_thresh=dlist[0]
upper_thresh=dlist[-1]
nb_classes=upper_thresh-lower_thresh+1
print("Number of classes:",nb_classes)
dim=64

def preprocessing(_path):
    '''
        preprocesses an image and label
        preprocess ops:
            * data shuffle
        image ops:
            * read
            * resize
            * thresh
            * image to tensor
            * convert to float (special case)
        label ops:
            * extract label from path
            * convert to float (special case)
        
        args:
            _path = the path of test dir / train dir where all the images are separated with corresponding labels
    '''
    X=[]
    y=[]
    img_paths=glob(os.path.join(_path,'*/*.bmp'))
    # shuffle
    random.shuffle(img_paths)
    # iterate
    for img_path in tqdm(img_paths[:5]):
        # extract int label from full path
        label=int(os.path.basename(os.path.dirname(img_path)))-lower_thresh
        # encode label
        encoded_label=np.zeros(nb_classes,dtype=np.float)
        encoded_label[label]=1
        # update
        y.append(encoded_label)

        # image ops
        # read
        img=cv2.imread(img_path,0)
        # resize
        img=cv2.resize(img,(dim,dim))
        # Otsu's thresholding after Gaussian filtering
        blur = cv2.GaussianBlur(img,(5,5),0)
        _,img = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
        # image to tensor
        x=np.expand_dims(img,axis=-1)
        x=np.expand_dims(x,axis=0)
        # float conversion
        x=x/255.0
        x=1-x
        # debug
        #plt.imshow(np.squeeze(x))
        #plt.show()
        # update
        X.append(x)
        
    # stacking
    X=np.vstack(X)
    y=np.vstack(y)
    return X,y
x_test,y_test=preprocessing(test_path)
print(x_test.shape,y_test.shape)

Number of classes: 50


  0%|          | 0/5 [00:00<?, ?it/s]

(5, 64, 64, 1) (5, 50)
